# Universidad de Antioquia
## Especialización en Analítica y Ciencia de Datos

Integrantes:

Jose Daniel Alvear Acevedo

Alfonso Cubillos Delgado

In [ ]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/Three_Phase_Electric_Power_Transmission.jpg/1200px-Three_Phase_Electric_Power_Transmission.jpg')

# **Contexto del problema**

Las líneas eléctricas aéreas de media tensión recorren cientos de millas para suministrar energía a las ciudades. Estas grandes distancias hacen que sea costoso inspeccionar manualmente las líneas en busca de daños que no provoquen inmediatamente un corte de energía, como una rama de un árbol golpeando la línea o una falla en el aislante. Estos modos de daño conducen a un fenómeno conocido como descarga parcial, una descarga eléctrica que no une completamente los electrodos entre un sistema de aislamiento. Las descargas parciales dañan lentamente la línea eléctrica, por lo que si no se reparan, eventualmente provocarán un corte de energía o provocarán un incendio.

Su desafío es detectar patrones de descarga parcial en señales adquiridas de estas líneas eléctricas con un nuevo medidor diseñado en el Centro ENET en VŠB . Los clasificadores efectivos que utilizan estos datos permitirán monitorear continuamente las líneas eléctricas en busca de fallas.

El Centro ENET investiga y desarrolla recursos energéticos renovables con el objetivo de reducir o eliminar los impactos ambientales nocivos. Sus esfuerzos se centran en desarrollar soluciones tecnológicas en torno al transporte y procesamiento de materias primas energéticas.

Al desarrollar una solución para detectar descargas parciales, ayudará a reducir los costos de mantenimiento y evitará cortes de energía.

# **Importanto librerias**

In [ ]:
import numpy as np 
import pandas as pd 
import pyarrow.parquet as pq # Leer archivos parquet
import matplotlib.pyplot as plt
import os
import seaborn as sns

# **Tomamos las 2000 primeras filas**

In [ ]:
INIT_DIR = '../input'
tamaño = 2001

In [ ]:
train = pq.read_pandas(os.path.join(INIT_DIR, 'vsb-power-line-fault-detection/train.parquet'), columns=[str(i) for i in range(tamaño)]).to_pandas()
metadata = pd.read_csv('../input/vsb-power-line-fault-detection/metadata_train.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
metadata.head()

In [ ]:
metadata.shape

In [ ]:
train_metadata = metadata[:tamaño]
train_metadata

In [ ]:
train_metadata.shape

Cada columna representa una señal, por lo tanto para manejar el mismo formato de pandas, trasponemos la base de datos

In [ ]:
train = train.T
train.shape

In [ ]:
train.head(5)

Adicionamos el Id de la señal en el dataframe principal

In [ ]:
train['signal_id'] = list(train_metadata['signal_id'])

In [ ]:
train.head(5)

**Unión de metadatos y datos de señales basados en signal_id**

In [ ]:
train = train.merge(train_metadata, on='signal_id')

In [ ]:
train.head(5)

**Revisando valores nulos en el dataframe**

In [ ]:
train.isnull().sum().sum()

**Gráficos de conteo frente a gráficos de destino para verificar el desequilibrio de datos**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
sns.countplot(x="target", data=train, ax=ax1)
sns.countplot(x="target", data=train, hue="phase", ax=ax2);

In [ ]:
target_count = train.target.value_counts()
print("negative(target=0) target: {}".format(target_count[0]))
print("positive(target=1) target: {}".format(target_count[1]))
print("positive data {:.3}%".format((target_count[1]/(target_count[0]+target_count[1]))*100))

In [ ]:
train[['id_measurement', 'phase']]

In [ ]:
target_mismatch = train[["id_measurement", "target"]].groupby(["id_measurement"]).sum().query("target != 3 & target != 0")
print("Valores de la variable objetivo que no son positivos ni negativos: {}".format(target_mismatch.shape[0]))
target_mismatch

Revisemos el Id == 67, donde la variable objetivo es diferente en diferentes fases

In [ ]:
train[train['id_measurement'] == 67]

**Observación:**

Las variables objetivo pueden ser diferentes para la misma señal en diferentes fases

**Hallamos el único valor del id_measurement**

In [ ]:
print("id_measurement tiene {} valores únicos".format(train.id_measurement.nunique()))

**Descripción de la columna id_measurement**

In [ ]:
train.id_measurement.value_counts().describe()

**Valores únicos de la columna phase**

In [ ]:
print("La variable fase tiene {} valores únicos en train {}".format(len(train.phase.unique()),train.phase.unique()))

In [ ]:
sns.countplot(train['phase']);

# **Graficando las señales**

**Graficando una señal normal**

In [ ]:
train.loc[1]['target']

In [ ]:
plt.figure(figsize=(24, 8))
plt.plot((train.loc[1].values), alpha=0.7);
plt.ylim([-100, 100])

**Grficando una señal defectuosa**

In [ ]:
train.loc[201]['target']

In [ ]:
plt.figure(figsize=(24, 8))
plt.plot((train.loc[201].values), alpha=0.7);
plt.ylim([-100, 100])

**Graficando las tres fases de una señal**

In [ ]:
train.loc[0:2][['target', 'id_measurement']]

In [ ]:
plt.figure(figsize=(24, 8))
plt.plot((train.loc[0].values), alpha=0.7);
plt.plot((train.loc[1].values), alpha=0.7);
plt.plot((train.loc[2].values), alpha=0.7);
plt.ylim([-100, 100])

**Graficando las tres fases de una señal defectuosa**

In [ ]:
train.loc[3:5][['target', 'id_measurement']]

In [ ]:
plt.figure(figsize=(24, 8))
plt.plot((train.loc[3].values), alpha=0.7);
plt.plot((train.loc[4].values), alpha=0.7);
plt.plot((train.loc[5].values), alpha=0.7);
plt.ylim([-100, 100])

# **Primera Iteración**

Dado el tamaño de los datos, se recomienda reiniciar el entorno para ejecutar las sigientes líneas.

In [ ]:
import os
print(os.listdir("../input"))

Cargamos los datos nuevamente

In [ ]:
train_data = pq.read_pandas(os.path.join(INIT_DIR, 'vsb-power-line-fault-detection/train.parquet')).to_pandas()
train_metadata = pd.read_csv('../input/vsb-power-line-fault-detection/metadata_train.csv')

In [ ]:
train_metadata.shape

In [ ]:
train_metadata.head(10)

In [ ]:
train_data.shape

In [ ]:
train_data.head(10)

Vamos a tomar el primer 1% de los datos para realizar una primera iteración del modelo, ya que no poseemos recursos computacionales para poder tomar la base de datos completa.

In [ ]:
train = train_data.iloc[:8000]
target = train_metadata.target[:8000]
print(train.shape)
print(target.shape)
train

In [ ]:
train.info()

A continuación realizaremos un análisis de componentes principales (PCA por sus siglas en inglés) para determinar si podemos utilizar una menor cantidad de variables.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.5, whiten=True)

X_pca = pca.fit_transform(train)

print('Número original de atributos:', train.shape[1])
print('Número reducido de atributos:', X_pca.shape[1])

Con el análisis previo, vamos a implementar inicialmente una regresión logística como primera iteración del conjunto de datos.

Para ello, vamos a utilizar la regresión logística con los datos originales, con el PCA y los datos estandarizados.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.3, random_state=42)

model = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Precisión de datos originales:', accuracy_score(y_test, y_pred))
#######

X_train, X_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.3, random_state=42)

model = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=100)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Precisión de datos reducidos:', accuracy_score(y_test, y_pred))

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.3, random_state=42)

pca = PCA(n_components=0.5, whiten=True)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=100)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Precisión de datos originales estandarizados:', accuracy_score(y_test, y_pred))

# **Plan de trabajo para las siguuientes iteraciones**

De los anteriores resultados, podemos observar que se obtuvo una precisión muy alta en los tres modelos. Por una análisis posterior podemos afirmar que el modelo se ve afectado por el desbalanceo de los datos, por lo tanto el conjunto de entrenamiento y validación no se están particionando de forma adecuada.

Así, para las siguientes iteraciones vamos a aplicar técnicas para el análisis de datos desbalanceados tales como: submuestreo inteligente, sobremuestreo inteligente, muestreo durante la validación y pesos diferentes para el error durante el entrenamiento.

Además determinar si existe independencia entre las muestras para así poder realizar un análisis suponiendo normalidad de los datos y poder tomar muestras las cuales no van a interferir con las demás.       